# Template matching parameter optimisation
Testing an option for finding template matcing parameters by optimisation of the correlation between an experimental pattern of known phase and orientation and the corresponding, simulated pattern.

In [ ]:
%matplotlib qt
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds

import matplotlib.pyplot as plt

import pyxem as pxm
from pyxem.utils import correlate
from pyxem.generators.indexation_generator import IndexationGenerator
from pyxem.generators.library_generator import DiffractionLibraryGenerator
from pyxem.generators.diffraction_generator import DiffractionGenerator
from pyxem.generators.structure_library_generator import StructureLibraryGenerator
from pyxem.libraries.diffraction_library import load_DiffractionLibrary

import hyperspy.api as hs
from hyperspy.component import Component
import diffpy.structure
from transforms3d.euler import euler2mat


In [ ]:
data_dir = 'D:/Dokumenter/MTNANO/Prosjektoppgave/SPED_data_GaAs_NW/'
structure_zb_file = 'D:\\Dokumenter/MTNANO/Prosjektoppgave/Data/Gen/NN_test_data/GaAs_mp-2534_conventional_standard.cif'
structure_wz_file = 'D:\\Dokumenter/MTNANO/Prosjektoppgave/Data/Gen/NN_test_data/GaAs_mp-8883_conventional_standard.cif'

#s = hs.load(data_dir + 'raw/Julie_180510_SCN45_FIB_c.blo', lazy=True)
s = hs.load(data_dir + 'gen/Julie_180510_SCN45_FIB_a_pyxem_sample.hdf5', lazy=True)


#s = s.inav[100:105, 0:1810]
#s = s.inav[100:101, 150:151]             # a.blo ZB 1
#orientation = np.deg2rad([90, 45, 173])  # ZB 1 a.blo
s = s.inav[100:101, 80:81]               # a.blo ZB 2
orientation = np.deg2rad([90, 45, 103])  # ZB 2 a.blo
#s = s.inav[50:51, 200:201]                  # a.blo WZ
#orientation = np.deg2rad([120, 90, 140.5])  # WZ a.blo
#s = s.inav[100:101, 1200:1201]            # ZB c.blo
#orientation = np.deg2rad([129, 35, 144])  # ZB c.blo
structure_file = structure_zb_file


s.change_dtype('float64')
s.data /= s.data.max()
s = pxm.ElectronDiffraction(s)
s = s.remove_background('gaussian_difference', sigma_min=2, sigma_max=8)
half_pattern_size = s.data.shape[2] // 2
structure = diffpy.structure.loadStructure(structure_file)
beam_energy = 200

matrix = euler2mat(orientation[0], orientation[1], orientation[2], 'rzxz')
latt_rot = diffpy.structure.lattice.Lattice(structure.lattice.a, structure.lattice.b, structure.lattice.c, structure.lattice.alpha, structure.lattice.beta, structure.lattice.gamma, baserot=matrix)
structure.placeInLattice(latt_rot)

In [ ]:
import time
def correlate_with_params(params, dp, structure, half_pattern_size, beam_energy):
    #start = time.time()
    reciprocal_angstrom_per_pixel, max_excitation_error, scale_x, scale_y, offset_x, offset_y = params
    reciprocal_radius = reciprocal_angstrom_per_pixel*(half_pattern_size - 1)
    
    image = dp.apply_affine_transformation(
        np.array([
            [scale_x, 0, offset_x],
            [0, scale_y, offset_y],
            [0, 0, 1]
        ]),
        inplace=False,
        show_progressbar=False).data[0, 0]

    generator = pxm.DiffractionGenerator(beam_energy, max_excitation_error=max_excitation_error)
    simulation = generator.calculate_ed_data(structure,
                                             reciprocal_radius,
                                             with_direct_beam=False)
    pattern_intensities = np.log(1 + simulation.intensities)
    
    #print(half_pattern_size)
    simulation.calibration = reciprocal_angstrom_per_pixel
    #print(simulation.calibrated_coordinates[0:2, :2])
    pixel_coordinates = np.rint(
        simulation.calibrated_coordinates[:, :2] + half_pattern_size).astype(int)
    image_intensities = image[pixel_coordinates[:, 1], pixel_coordinates[:, 0]]
    #end = time.time()
    #print('.', end='')
    #print(end - start, len(simulation.intensities))
    #pattern_norm = np.sqrt(np.dot(pattern_intensities,
                            #pattern_intensities))
    pattern_norm = np.linalg.norm(pattern_intensitites)
    if False:
        plt.figure('test')
        plt.cla()
        plt.imshow(image)
        plt.scatter(pixel_coordinates[:, 0], pixel_coordinates[:, 1], marker='x', c=pattern_intensities, cmap='autumn_r')
        plt.show()

    #print(-np.dot(image_intensities, pattern_intensities) / pattern_norm,
    #     reciprocal_angstrom_per_pixel, max_excitation_error, scale_x, scale_y, offset_x, offset_y)
    return -np.dot(image_intensities, pattern_intensities) / pattern_norm

In [ ]:
scale_x = 1
scale_y = 1
offset_x = 0
offset_y = 0
reciprocal_angstrom_per_pixel = 0.033
s.set_diffraction_calibration(reciprocal_angstrom_per_pixel)
max_excitation_error = 1.0 / 8.0
initial_guess = np.array((
    reciprocal_angstrom_per_pixel,
    max_excitation_error,
    scale_x,
    scale_y,
    offset_x,
    offset_y))
bounds = Bounds([0.030, 0.1, 0.8, 0.8, -2, -2], [0.05, 0.5, 1.2, 1.2, 2, 2])
result = minimize(correlate_with_params, initial_guess,
                  args=(
                      s,
                      structure,
                      half_pattern_size,
                      #max_excitation_error,
                      beam_energy),
                  #method='Nelder-Mead',
                  method='TNC',
                  bounds=bounds,
                  options={
                      'disp': True,
                      'maxiter': 500
                  })

In [ ]:
opt_reciprocal_angstrom_per_pixel,\
opt_max_excitation_error,\
opt_scale_x,\
opt_scale_y,\
opt_offset_x,\
opt_offset_y = result['x']

In [ ]:
print('rec =', opt_reciprocal_angstrom_per_pixel, reciprocal_angstrom_per_pixel)
print('max_excitation_error = ', opt_max_excitation_error, max_excitation_error, 1/opt_max_excitation_error)
print('scale_x =', opt_scale_x, scale_x)
print('scale_y =', opt_scale_y, scale_y)
print('offset_x =', opt_offset_x, offset_x)
print('offset_y =', opt_offset_y, offset_y)

In [ ]:
image = s.apply_affine_transformation(
        np.array([
            [opt_scale_x, 0, opt_offset_x],
            [0, opt_scale_y, opt_offset_y],
            [0, 0, 1]
        ]),
        inplace=False,
        show_progressbar=False).data[0, 0]
#image = s.data[0, 0]
reciprocal_radius = reciprocal_angstrom_per_pixel*(half_pattern_size - 1)
generator = pxm.DiffractionGenerator(beam_energy, max_excitation_error=opt_max_excitation_error)

simulation = generator.calculate_ed_data(structure,
                                         reciprocal_radius,
                                         with_direct_beam=False)
#simulation.as_signal(144, 0.04, reciprocal_radius).plot()
pattern_intensities = simulation.intensities

simulation.calibration = reciprocal_angstrom_per_pixel
pixel_coordinates = np.rint(
    simulation.calibrated_coordinates[:, :2] + half_pattern_size).astype(int)
plt.figure('Optimized')
plt.clf()
plt.imshow(image)
plt.scatter(pixel_coordinates[:, 0], pixel_coordinates[:, 1], marker='x', c=pattern_intensities, cmap='autumn_r')
plt.show()